In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv
/kaggle/input/word2vec-nlp-tutorial/testData.tsv
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv


# Part I   bag-of-words

## Reading the Data

The necessary files can be downloaded from the Data page. The first file that you'll need is unlabeledTrainData, which contains 25,000 IMDB movie reviews, each with a positive or negative sentiment label.

Next, read the tab-delimited file into Python. To do this, we can use the pandas package, introduced in the Titanic tutorial, which provides the read_csv function for easily reading and writing data files. If you haven't used pandas before, you may need to install it.

In [0]:
train = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv',header=0,delimiter='\t',quoting=3)
# header:第几行作为标题；delimiter:tab符号分隔（tsv:tab-seperated values document）；quoting:是否计入引号
train.head()

Here, "header=0" indicates that the first line of the file contains column names, "delimiter=\t" indicates that the fields are separated by tabs, and quoting=3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

We can make sure that we read 25,000 rows and 3 columns as follows:

In [0]:
train.shape

In [0]:
train.columns.values

There are HTML tags such as "
", abbreviations, punctuation - all common issues when processing text from online. Take some time to look through other reviews in the training set while you're at it - the next section will deal with how to tidy up the text for machine learning.

Removing HTML Markup: The BeautifulSoup Package
First, we'll remove the HTML tags. For this purpose, we'll use the Beautiful Soup library. 

In [0]:
from bs4 import BeautifulSoup
# 是一个可以从HTML或XML文件中提取数据的Python库

In [0]:
import re

In [0]:
import nltk
nltk.download('all')

In [0]:
from nltk.corpus import stopwords

Calling get_text() gives you the text of the review, without tags or markup. If you browse the BeautifulSoup documentation, you'll see that it's a very powerful library - more powerful than we need for this dataset. However, it is not considered a reliable practice to remove markup using regular expressions, so even for an application as simple as this, it's usually best to use a package like BeautifulSoup.

Dealing with Punctuation, Numbers and Stopwords: NLTK and regular expressions
When considering how to clean the text, we should think about the data problem we are trying to solve. For many problems, it makes sense to remove punctuation. On the other hand, in this case, we are tackling a sentiment analysis problem, and it is possible that "!!!" or ":-(" could carry sentiment, and should be treated as words. In this tutorial, for simplicity, we remove the punctuation altogether, but it is something you can play with on your own.

Similarly, in this tutorial we will remove numbers, but there are other ways of dealing with them that make just as much sense. For example, we could treat them as words, or replace them all with a placeholder string such as "NUM".

To remove punctuation and numbers, we will use a package for dealing with regular expressions, called re. The package comes built-in with Python; no need to install anything. For a detailed description of how regular expressions work, see the package documentation. 

A full overview of regular expressions is beyond the scope of this tutorial, but for now it is sufficient to know that [] indicates group membership and ^ means "not". In other words, the re.sub() statement above says, "Find anything that is NOT a lowercase letter (a-z) or an upper case letter (A-Z), and replace it with a space."

We'll also convert our reviews to lower case and split them into individual words (called "tokenization" in NLP lingo):

Finally, we need to decide how to deal with frequently occurring words that don't carry much meaning. Such words are called "stop words"; in English they include words such as "a", "and", "is", and "the". Conveniently, there are Python packages that come with stop word lists built in. Let's import a stop word list from the Python Natural Language Toolkit (NLTK). You'll need to install the library if you don't already have it on your computer; you'll also need to install the data packages that come with it, as follows:

In [0]:
def preprocessing_review(raw_review):
    #remove html
    text = BeautifulSoup(raw_review).get_text()
    #remove punctuation and numbers,change to lower case and split.
    words = re.sub('[^a-zA-Z]',' ',text).lower().split() 
    stop = set(stopwords.words('english'))
    # In Python, searching a set is much faster than searching
    review = [w for w in words if w not in stop and len(w)!=1]
    return ' '.join(review)

Two elements here are new: First, we converted the stop word list to a different data type, a set. This is for speed; since we'll be calling this function tens of thousands of times, it needs to be fast, and searching sets in Python is much faster than searching lists.

Second, we joined the words back into one paragraph. This is to make the output easier to use in our Bag of Words, below. 

In [0]:
review_list = []
for i in range(len(train)):
    review_list.append(preprocessing_review(train['review'][i]))

In [0]:
review_list

## Creating Features from a Bag of Words (Using scikit-learn)

Now that we have our training reviews tidied up, how do we convert them to some kind of numeric representation for machine learning? One common approach is called a Bag of Words. The Bag of Words model learns a vocabulary from all of the documents, then models each document by counting the number of times each word appears. For example, consider the following two sentences:

Sentence 1: "The cat sat on the hat"

Sentence 2: "The dog ate the cat and the hat"

From these two sentences, our vocabulary is as follows:

{ the, cat, sat, on, hat, dog, ate, and }

To get our bags of words, we count the number of times each word occurs in each sentence. In Sentence 1, "the" appears twice, and "cat", "sat", "on", and "hat" each appear once, so the feature vector for Sentence 1 is:

{ the, cat, sat, on, hat, dog, ate, and }

Sentence 1: { 2, 1, 1, 1, 1, 0, 0, 0 }

Similarly, the features for Sentence 2 are: { 3, 1, 0, 0, 1, 1, 1, 1}

In the IMDB data, we have a very large number of reviews, which will give us a large vocabulary. To limit the size of the feature vectors, we should choose some maximum vocabulary size. Below, we use the 5000 most frequent words (remembering that stop words have already been removed).

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
# 词袋模型
vectorizer = CountVectorizer(analyzer='word',max_features=5000)
train_feature = vectorizer.fit_transform(review_list)

In [0]:
train_feature = train_feature.toarray()

In [0]:
vectorizer.get_feature_names()

## Random Forest

At this point, we have numeric training features from the Bag of Words and the original sentiment labels for each feature vector, so let's do some supervised learning! Here, we'll use the Random Forest classifier that we introduced in the Titanic tutorial.  The Random Forest algorithm is included in scikit-learn (Random Forest uses many tree-based classifiers to make predictions, hence the "forest"). Below, we set the number of trees to 100 as a reasonable default value. More trees may (or may not) perform better, but will certainly take longer to run. Likewise, the more features you include for each review, the longer this will take.

In [0]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)

In [0]:
forest = forest.fit(train_feature,train['sentiment'])

## Creating a Submission

All that remains is to run the trained Random Forest on our test set and create a submission file. If you haven't already done so, download testData.tsv from the Data page. This file contains another 25,000 reviews and ids; our task is to predict the sentiment label.

Note that when we use the Bag of Words for the test set, we only call "transform", not "fit_transform" as we did for the training set. In machine learning, you shouldn't use the test set to fit your model, otherwise you run the risk of overfitting. For this reason, we keep the test set off-limits until we are ready to make predictions.

In [0]:
test = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/testData.tsv',header=0,delimiter='\t',quoting=3)
test.head()

In [0]:
test_review_list = []
for i in range(len(test)):
    test_review_list.append(preprocessing_review(test['review'][i]))

In [0]:
test_feature = vectorizer.transform(test_review_list)
test_feature = test_feature.toarray()
test_feature

In [0]:
result = forest.predict(test_feature)

In [0]:
result_pd = pd.DataFrame(data={'id':test['id'],'sentiment':result})

In [0]:
result_pd

In [0]:
result_pd.to_csv('submission.csv',index=False,quoting=3)

Try different things and see how your results change. You can clean the reviews differently, choose a different number of vocabulary words for the Bag of Words representation, try Porter Stemming, a different classifier, or any number of other things. 

# Part II   word2vec

## Introducing Distributed Word Vectors

This part of the tutorial will focus on using distributed word vectors created by the Word2Vec algorithm. 

Parts 2 and 3 assume more familiarity with Python than Part 1.

Word2vec, published by Google in 2013, is a neural network implementation that learns distributed representations for words. Other deep or recurrent neural network architectures had been proposed for learning word representations prior to this, but the major problem with these was the long time required to train the models. Word2vec learns quickly relative to other models.

Word2Vec does not need labels in order to create meaningful representations. This is useful, since most data in the real world is unlabeled. If the network is given enough training data (tens of billions of words), it produces word vectors with intriguing characteristics. Words with similar meanings appear in clusters, and clusters are spaced such that some word relationships, such as analogies, can be reproduced using vector math. The famous example is that, with highly trained word vectors, "king - man + woman = queen."

Check out Google's code, writeup, and the accompanying papers. This presentation is also helpful. The original code is in C, but it has since been ported to other languages, including Python. We encourage you to play with the original C tool, but be warned that it is not user-friendly if you are a beginning programmer (we had to manually edit the header files to compile it).

Recent work out of Stanford has also applied deep learning to sentiment analysis; their code is available in Java. However, their approach, which relies on sentence parsing, cannot be applied in a straightforward way to paragraphs of arbitrary length.

Distributed word vectors are powerful and can be used for many applications, particularly word prediction and translation. Here, we will try to apply them to sentiment analysis.

## Using word2vec in Python

In Python, we will use the excellent implementation of word2vec from the gensim package.

Although Word2Vec does not require graphics processing units (GPUs) like many deep learning algorithms, it is compute intensive. Both Google's version and the Python version rely on multi-threading (running multiple processes in parallel on your computer to save time). ln order to train your model in a reasonable amount of time, you will need to install cython (instructions here). Word2Vec will run without cython installed, but it will take days to run instead of minutes.

In [2]:
unlabeltrain = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv',header=0,delimiter='\t',quoting=3)
train = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv',header=0,delimiter='\t',quoting=3)
test = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/testData.tsv',header=0,delimiter='\t',quoting=3)

In [0]:
import nltk
nltk.download('all')

In [3]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [4]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def preprocessing_review(raw_review,tokenizer,remove_stopwords=False):
    #remove html
    text = BeautifulSoup(raw_review).get_text()
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentence = []
    for word in raw_sentences:
        if len(word) > 0:
            #remove punctuation and numbers,change to lower case and split.
            words = re.sub('[^a-zA-Z0-9]',' ',word).lower().split() 
            if remove_stopwords:
                stops = set(stopwords.words('english'))
                review = [w for w in words if w not in stops]
            else:
                review = [w for w in words]
            sentence.extend(review)
    return sentence

In [5]:
sentences = []

for i in train['review']:
    sentences.append(preprocessing_review(i,tokenizer))
for i in unlabeltrain['review']:
    sentences.append(preprocessing_review(i,tokenizer))

In [6]:
sentences

[['with',
  'all',
  'this',
  'stuff',
  'going',
  'down',
  'at',
  'the',
  'moment',
  'with',
  'mj',
  'i',
  've',
  'started',
  'listening',
  'to',
  'his',
  'music',
  'watching',
  'the',
  'odd',
  'documentary',
  'here',
  'and',
  'there',
  'watched',
  'the',
  'wiz',
  'and',
  'watched',
  'moonwalker',
  'again',
  'maybe',
  'i',
  'just',
  'want',
  'to',
  'get',
  'a',
  'certain',
  'insight',
  'into',
  'this',
  'guy',
  'who',
  'i',
  'thought',
  'was',
  'really',
  'cool',
  'in',
  'the',
  'eighties',
  'just',
  'to',
  'maybe',
  'make',
  'up',
  'my',
  'mind',
  'whether',
  'he',
  'is',
  'guilty',
  'or',
  'innocent',
  'moonwalker',
  'is',
  'part',
  'biography',
  'part',
  'feature',
  'film',
  'which',
  'i',
  'remember',
  'going',
  'to',
  'see',
  'at',
  'the',
  'cinema',
  'when',
  'it',
  'was',
  'originally',
  'released',
  'some',
  'of',
  'it',
  'has',
  'subtle',
  'messages',
  'about',
  'mj',
  's',
  'feeling'

You may get a few warnings from BeautifulSoup about URLs in the sentences. These are nothing to worry about (although you may want to consider removing URLs when cleaning the text). 

## Training and Saving Your Model

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

- Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
- Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
- Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
- Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
- Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
- Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
- Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [7]:
from gensim.models import word2vec
model = word2vec.Word2Vec(sentences,workers=4,size=300,min_count=40,window=10,sample=1e-3)

In [8]:
model.wv.syn0.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(16731, 300)

In [9]:
model['the']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.04624734,  0.37996936,  0.6620191 ,  0.54243034, -1.1419032 ,
        0.14235386,  0.9466752 ,  1.7273223 ,  0.46808907, -0.90142787,
        0.9664341 ,  1.1155123 , -0.6827945 , -1.0313067 ,  1.2572081 ,
        0.19455592,  0.11226726,  0.1235632 ,  0.17669123,  0.05197305,
        0.15678175, -1.097783  , -0.2724593 ,  0.7264795 ,  0.27921638,
       -0.43149886,  0.185892  ,  0.3408598 ,  0.26591843, -1.3996001 ,
       -0.8350427 , -0.30081695, -1.4344863 ,  0.72916144,  1.5093292 ,
        0.05635867,  0.9933831 , -0.3846225 , -0.26076773, -0.07276369,
       -0.6695548 , -0.17088783, -0.54777044,  0.2298524 ,  1.7141536 ,
       -0.560334  ,  0.05889451,  0.1360954 ,  0.5939756 ,  1.6230795 ,
        0.12466273, -0.7961184 ,  0.18021776,  0.11042725,  0.7897062 ,
        0.48836946, -0.03503352, -0.21015729, -0.20956998, -0.44876847,
        0.894087  ,  0.06192057, -0.9333796 ,  1.1796366 , -0.07907875,
        0.9402492 , -0.5273074 ,  0.3800571 , -0.88861805,  0.26

# Part III   word2vec(averaging)

## Numeric Representations of Words

Now that we have a trained model with some semantic understanding of words, how should we use it? If you look beneath the hood, the Word2Vec model trained in Part 2 consists of a feature vector for each word in the vocabulary, stored in a numpy array called "syn0":

The number of rows in syn0 is the number of words in the model's vocabulary, and the number of columns corresponds to the size of the feature vector, which we set in Part 2.  Setting the minimum word count to 40 gave us a total vocabulary of 16,492 words with 300 features apiece.

One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

In [0]:
def avg_feature_vec(sentence,model,num_features):
    index2word = set(model.wv.index2word)
    sum_word = np.zeros(num_features)
    for word in sentence:
        if word in index2word:
            sum_word += np.array(model[word])
    sum_word = sum_word / len(sentence)
    return sum_word.tolist()

In [15]:
clean_train_reviews = []
for i in train['review']:
    clean_train_reviews.append(preprocessing_review(i,tokenizer,remove_stopwords=True))

In [16]:
clean_test_reviews = []
for i in test['review']:
    clean_test_reviews.append(preprocessing_review(i,tokenizer,remove_stopwords=True))

In [0]:
train_avg_features = []
for i in clean_train_reviews:
    train_avg_features.append(avg_feature_vec(i,model,300))
train_avg_features = np.array(train_avg_features)

In [0]:
test_avg_features = []
for i in clean_test_reviews:
    test_avg_features.append(avg_feature_vec(i,model,300))
test_avg_features = np.array(test_avg_features)

In [0]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)

In [0]:
forest.fit(train_avg_features,train['sentiment'])

In [0]:
result = forest.predict(test_avg_features)

In [0]:
result_pd = pd.DataFrame(data={'id':test['id'],'sentiment':result})
result_pd.to_csv('submission.csv',index=False,quoting=3)

We found that this produced results much better than chance, but underperformed Bag of Words by a few percentage points.

Since the element-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way? A standard way of weighting word vectors is to apply "tf-idf" weights, which measure how important a given word is within a given set of documents. One way to extract tf-idf weights in Python is by using scikit-learn's TfidfVectorizer, which has an interface similar to the CountVectorizer that we used in Part 1. However, when we tried weighting our word vectors in this way, we found no substantial improvement in performance.

# Part III   word2vec(clustering)

Word2Vec creates clusters of semantically related words, so another possible approach is to exploit the similarity of words within a cluster. Grouping vectors in this way is known as "vector quantization." To accomplish this, we first need to find the centers of the word clusters, which we can do by using a clustering algorithm such as K-Means.

In K-Means, the one parameter we need to set is "K," or the number of clusters. How should we decide how many clusters to create? Trial and error suggested that small clusters, with an average of only 5 words or so per cluster, gave better results than large clusters with many words. Clustering code is given below. We use scikit-learn to perform our K-Means.

In [10]:
from sklearn.cluster import KMeans

In [11]:
num_clusters = model.wv.syn0.shape[0] // 5

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


The cluster assignment for each word is now stored in idx, and the vocabulary from our original Word2Vec model is still stored in model.index2word. For convenience, we zip these into one dictionary as follows:

In [12]:
km_clustering = KMeans(n_clusters=num_clusters)
idx = km_clustering.fit_predict(model.wv.syn0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [13]:
word_centroid_map = dict(zip(model.wv.index2word,idx))

In [ ]:
for i in range(10):
    print("\nCluster %d" % i)
    words = []
    for j in range(0,len(word_centroid_map.values())):
        if(list(word_centroid_map.values())[j]==i):
            words.append(list(word_centroid_map.keys())[j])
    print(words)

At any rate, now we have a cluster (or "centroid") assignment for each word, and we can define a function to convert reviews into bags-of-centroids. This works just like Bag of Words but uses semantically related clusters instead of individual words:

In [14]:
def create_bag_of_centroids(wordlist,word_centroid_map):
    num_centroids = max(word_centroid_map.values())+1
    bag_of_centroids = np.zeros(num_centroids,dtype='float32')
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

The function above will give us a numpy array for each review, each with a number of features equal to the number of clusters. Finally, we create bags of centroids for our training and test set, then train a random forest and extract results:

In [17]:
train_centroids = np.zeros((train['review'].size,num_clusters),dtype='float32')

counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review,word_centroid_map)
    counter += 1
    

test_centroids = np.zeros((test['review'].size,num_clusters),dtype='float32')

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review,word_centroid_map)
    counter += 1

fit a random forest and extract predictions

In [18]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(train_centroids,train['sentiment'])
result = forest.predict(test_centroids)
result_pd = pd.DataFrame(data={'id':test['id'],'sentiment':result})
result_pd.to_csv('submission.csv',index=False,quoting=3)

You may ask: Why is Bag of Words better?
The biggest reason is, in our tutorial, averaging the vectors and using the centroids lose the order of words, making it very similar to the concept of Bag of Words. The fact that the performance is similar (within range of standard error) makes all three methods practically equivalent.  

A few things to try:
First, training Word2Vec on a lot more text should greatly improve performance. Google's results are based on word vectors that were learned out of more than a billion-word corpus; our labeled and unlabeled training sets together are only a measly 18 million words or so. Conveniently, Word2Vec provides functions to load any pre-trained model that is output by Google's original C tool, so it's also possible to train a model in C and then import it into Python.

Second, in published literature, distributed word vector techniques have been shown to outperform Bag of Words models. In this paper, an algorithm called Paragraph Vector is used on the IMDB dataset to produce some of the most state-of-the-art results to date. In part, it does better than the approaches we try here because vector averaging and clustering lose the word order, whereas Paragraph Vectors preserves word order information.